In [119]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, ForeignKey, Integer, String, Float, DateTime,\
    Boolean
from sqlalchemy.orm import relationship

from datetime import datetime


SQLALCHEMY_DATABASE_URL = "sqlite:///./sql_app.db"


engine = create_engine(SQLALCHEMY_DATABASE_URL,
                       connect_args={"check_same_thread": False} # for sqlite
                       )
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()




from sqlalchemy.orm import Session
from fastapi import Depends


class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String(20), index=True, unique=True)
    email = Column(String, unique=True)
    hashed_password = Column(String, nullable=False)
    is_active = Column(Boolean, default=True)
    is_superuser = Column(Boolean, default=False)
    items = relationship("Item", back_populates="owner")

    
class Item(Base):
    __tablename__ = "items"
    id = Column(Integer, primary_key=True, index=True)
    label = Column(String)
    owner_id = Column(Integer, ForeignKey(User.id))
    owner = relationship("User", back_populates="items")
    
    def __str__(self):
        return 'Item id: '+str(self.id)

Base.metadata.create_all(bind=engine)
    
def get_db():
    db = SessionLocal()
    try:
        yield db
    except Exception as e:
        print('Error: ' + str(type(e)))
    finally:
        db.close()

In [120]:
from typing import Any, Dict, Optional, Union

def create_user(username: str, email:str, password: str ,db: Session)-> User:
    fake_hashed_password = password+ 'fake_hashed'
    db_user = User(username = username,
                   email = email,
                   hashed_password = password
                   )
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user


def get_user_with_username(username: str, db: Session)-> Optional[User]:
    return db.query(User).filter(User.username == username).first()


def get_user_with_email(email: str, db: Session)-> Optional[User]:
    return db.query(User).filter(User.email == email).first()


def set_active(username: str, state: bool, db: Session)-> bool:
    try:
        user = get_user_with_username(username, db)
        user.is_active = state
        db.commit()
        return True
    except:
        return False

    
def set_superuser(username: str, state: bool, db: Session)-> bool:
    try:
        user = get_user_with_username(username, db)
        user.is_superuser = state
        db.commit()
        return True
    except:
        return False

    
def is_active(username: str, db: Session)-> bool:
    user = get_user_with_username(username, db)
    return user.is_active


def delete_user(username: str, db: Session)-> bool:
    user = get_user_with_username(username, db)
    try:
        db.delete(user)
        db.commit()
        return True
    except:
        return False

In [87]:
u1 = create_user(username = "ju", email="ju", password="12345", db=SessionLocal())
u1.username, u1.email, u1.hashed_password, u1.is_active, u1.id

('ju', 'ju', '12345', True, 7)

In [89]:
get_user_with_username("ju", SessionLocal())


In [83]:
change_is_active_state("ju", False, SessionLocal())
u1 = get_user_with_username("ju", SessionLocal())
u1.is_active

False

In [86]:
delete_user("ju", SessionLocal())

True

In [129]:
def create_item(username: str, label:str,db: Session)-> Item:
    user = get_user_with_username(username, db)
    new_item = Item(label = label,
                    owner_id = user.id,
                   )
    db.add(new_item)
    db.commit()
    db.refresh(new_item)
    return new_item

def get_item(item_id: int, db: Session)->Item:
    return db.query(Item).filter(Item.id==item_id).first() 

def check_item_owner(item_id: int, owner_id: int, db: Session)-> bool:
    item = db.query(Item).filter(Item.id == item_id).first()
    return item.owner.id == owner_id


def edit_item(item_id: str, label:str, db: Session)-> Item:
    
    pass


def delete_item(item_id: int, db: Session)->bool:
    item = db.query(Item).filter(Item.id == item_id).first()
    try:
        db.delete(item)
        db.commit()
        return True
    except:
        return False



In [130]:
create_item("ju", "hol", SessionLocal())

In [131]:
get_item(1,SessionLocal()).owner.id

7

In [132]:
check_item_owner(1,7, SessionLocal())

True

In [99]:
u1 = get_user_with_username("ju", SessionLocal())
u1.items[1].label

'hol'

In [28]:
u1.username, u1.email, u1.hashed_password, u1.is_active

('juan1', 'juan@gmail1.com', '12345', True)

In [38]:
find_user_with_username("juans", db)

In [ ]:
def get_user_by_email(db: Session, email: str):
    return db.query(User).filter(User.email == email).first()


def get_user_by_username(db: Session, username: str):
    return db.query(User).filter(User.username == username).first()


def get_users(db: Session, skip: int = 0, limit: int = 100):
    return db.query(User).offset(skip).limit(limit).all()



def create_user(username: str, email:str, password: str ,db: Session)-> User:
    fake_hashed_password = password+ 'fake_hashed'
    db_user = User(username = username,
                   email = email,
                   hashed_password = password
                   )
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user
